In [1]:
import torch
from models import MVCNN
import time
import torchvision
from torchvision import transforms
import os

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [3]:
svcnn = MVCNN.SVCNN(name="VGG11_SVCNN")
svcnn_weights = torch.load(
    "trained-models/SVCNN/model-00010.pth", weights_only=True, map_location=device
)
svcnn.load_state_dict(svcnn_weights, strict=False)
svcnn.eval()
svcnn.to(device)

SVCNN(
  (net_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [4]:
random_out = None
random_input = torch.randn(12, 3, 244, 244).to(device)
with torch.no_grad():
    t1 = time.time()
    random_out = svcnn.net_1(random_input)
    print("Time taken for forward pass: ", time.time() - t1)

Time taken for forward pass:  0.07177281379699707


In [5]:
pooled_features, _ = torch.max(random_out, dim=0, keepdim=False)
pooled_features = pooled_features.view(1, -1)
pooled_features = pooled_features.to(device)
print(pooled_features.shape)

torch.Size([1, 25088])


In [6]:
with torch.no_grad():
    t2 = time.time()
    random_out_final = svcnn.net_2(pooled_features)
    print("Time taken for final forward pass: ", time.time() - t2)
random_out_final.shape
final_label = torch.argmax(random_out_final, dim=1)
print(final_label)

Time taken for final forward pass:  0.015916109085083008
tensor([9], device='mps:0')


In [7]:
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

In [8]:
objects = os.listdir("./ModelNet40-12View")
objects.remove('.DS_Store')
len(objects)

dataloaders = []

In [9]:
for object in objects:
    dataset = torchvision.datasets.ImageFolder(
        root=f'ModelNet40-12View/{object}/test',
        transform=transforms
    )
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=12, shuffle=False)
    dataloaders.append(dataloader)